In [2]:
from libray import *

In [3]:
from sklearn.neural_network import MLPClassifier

In [4]:
x_train, y_train,x_val , y_val, x_test , y_test , x_val_bal , y_val_bal , x_test_bal , y_test_bal, x_ml_classifer, x_ml_classifer_test = read_data()

In [52]:
x_train_gold, y_train_gold = read_data_gold_data_RF()

In [53]:
x_train.columns

Index(['CONSRANK_val', 'CP_HLPL', 'CP_MJ3h', 'DDG_V', 'CP_RMFCA',
       'AP_GOAP_DF'],
      dtype='object')

In [54]:
x_val[x_train.columns.to_list()]

,CONSRANK_val,CP_HLPL,CP_MJ3h,DDG_V,CP_RMFCA,AP_GOAP_DF
Conf,,,,,,
F_1EXB_1006_I.pdb,0.001748,0.736018,0.710550,0.782688,0.790792,0.721292
F_1EXB_1027_I.pdb,0.001116,0.543624,0.736239,0.583189,0.637449,0.721292
F_1EXB_1030_I.pdb,0.001695,0.791946,0.761927,0.824361,0.784954,0.721292
F_1EXB_1042_I.pdb,0.001909,0.749441,0.788991,0.647396,0.790252,0.721292
F_1EXB_1051_I.pdb,0.000283,0.653244,0.573394,0.710211,0.764343,0.721292
...,...,...,...,...,...,...
Z_2W9E_9945_I.pdb,0.102853,0.561521,0.456422,0.583773,0.678746,0.588665
Z_2W9E_9951_I.pdb,0.085397,0.512304,0.412385,0.629171,0.700889,0.628922
Z_2W9E_9958_I.pdb,0.049078,0.626398,0.418807,0.633153,0.678624,0.696119


In [55]:
x_train_gold.columns.to_list()

['CONSRANK_val',
 'CP_HLPL',
 'CP_MJ3h',
 'DDG_V',
 'CP_RMFCA',
 'AP_GOAP_DF',
 'TF2',
 'NNC']

In [56]:
PRC = MLPClassifier(activation='relu', alpha=3.6946012051993024e-06, batch_size=512,
                                hidden_layer_sizes=(24,), random_state=32,  solver='adam'
                               )
PCR_gold = MLPClassifier(activation='relu', alpha=3.6946012051993024e-06, batch_size=512,
                                hidden_layer_sizes=(24,), random_state=32,  solver='adam'
                               )

In [57]:
PRC.fit(X=x_train,y=y_train)

MLPClassifier(alpha=3.6946012051993024e-06, batch_size=512,
              hidden_layer_sizes=(24,), random_state=32)

In [58]:
PCR_gold.fit(X=x_train_gold,y=y_train_gold)

MLPClassifier(alpha=3.6946012051993024e-06, batch_size=512,
              hidden_layer_sizes=(24,), random_state=32)

In [12]:
scoring_functions = ['CONSRANK_val', 'CP_HLPL', 'CP_MJ3h', 'DDG_V', 'CP_RMFCA', 'AP_GOAP_DF']


In [13]:
# proba_to_be_true_core = pd.DataFrame(PRC.predict_proba(x_val[scoring_functions]))[1]
proba_to_be_true_core = pd.DataFrame(PRC.predict_proba(x_val[x_train.columns.to_list()]))[1]


In [14]:
# df_proba_true_core = pd.DataFrame([PRC.predict(x_val[scoring_functions]),proba_to_be_true_core]).T
df_proba_true_core = pd.DataFrame([PRC.predict(x_val[x_train.columns.to_list()]),proba_to_be_true_core]).T

In [15]:
x_val_gold = pd.concat( [x_val[scoring_functions],x_ml_classifer[["NNC","TF2"]]],axis=1)
x_test_gold = pd.concat( [x_test[scoring_functions],x_ml_classifer_test[["NNC","TF2"]]],axis=1)
#x_ml_classifer.head()

In [16]:
proba_to_be_true_haikal = pd.DataFrame(PCR_gold.predict_proba(x_val_gold))[1]


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- NNC
Feature names seen at fit time, yet now missing:
- RFC


In [ ]:
df_proba_true_haikal = pd.DataFrame([PCR_gold.predict(x_val_gold),proba_to_be_true_haikal]).T

In [28]:
def convert_pred_a(model , my_x, my_y, tag ):
    """ this funtions will get the columns neseray to create a dataframe to analize the success rate """
    df_pred = pd.DataFrame( model.predict_proba(my_x.to_numpy()) )
    df_pred = df_pred.set_index( my_x.index ) 
    label_binary = my_y
    idx = []

    if tag == "test":
        idx = [ x.split("_")[0] for x in my_x.index ]
        idx = pd.Series(idx, index=my_x.index)

    elif tag == "val":
        idx = [ x.split("_")[1] for x in my_x.index ]
        idx = pd.Series(idx, index=my_x.index)

    df_pred = pd.concat( [idx ,label_binary, df_pred[1] ], axis=1)
    df_pred.columns = ["idx","label_binary","proba_true"]
    
    return df_pred

In [ ]:
df_core_perceptron_val = convert_pred_a(model=PRC,my_x=x_val[[x_train.columns.to_list()]],my_y=y_val,tag="val")

/sw/csgv/machine_learning/2022.11/el7_cudnn8.2_cuda11.2_py3.8_env/machine_learning-module/env/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


In [ ]:
df_core_perceptron_val

,idx,label_binary,proba_true
Conf,,,
F_1EXB_1006_I.pdb,1EXB,False,0.020228
F_1EXB_1027_I.pdb,1EXB,False,0.034935
F_1EXB_1030_I.pdb,1EXB,False,0.016832
F_1EXB_1042_I.pdb,1EXB,False,0.038345
F_1EXB_1051_I.pdb,1EXB,False,0.021989
...,...,...,...
Z_2W9E_9945_I.pdb,2W9E,False,0.899762
Z_2W9E_9951_I.pdb,2W9E,False,0.780666
Z_2W9E_9958_I.pdb,2W9E,False,0.303802


In [ ]:
df_core_perceptron_test = convert_pred_a(model=PRC,my_x=x_test[[x_train.columns.to_list()]],my_y=y_test,tag="test")


/sw/csgv/machine_learning/2022.11/el7_cudnn8.2_cuda11.2_py3.8_env/machine_learning-module/env/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


In [ ]:
df_snorkel_perceptron_val = convert_pred_a(model=PCR_gold,my_x=x_val_gold,my_y=y_val,tag="val")
df_snorkel_perceptron_test = convert_pred_a(model=PCR_gold,my_x=x_test_gold,my_y=y_test,tag="test")


/sw/csgv/machine_learning/2022.11/el7_cudnn8.2_cuda11.2_py3.8_env/machine_learning-module/env/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/sw/csgv/machine_learning/2022.11/el7_cudnn8.2_cuda11.2_py3.8_env/machine_learning-module/env/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


In [ ]:
get_success_rate(df_core_perceptron_val,"val","NN")

NN [24, 28, 31, 42] val
43.64 50.91 56.36 76.36 


,NN
1,43.64
10,50.91
100,56.36
1000,76.36


In [ ]:
get_success_rate(df_core_perceptron_test,"test","NN")

NN [6, 7, 8, 13] test
40.0 46.67 53.33 86.67 


,NN
1,40.00
10,46.67
100,53.33
1000,86.67


In [ ]:
get_success_rate(df_snorkel_perceptron_val,"val","NN-hAIkal")

NN-hAIkal [24, 30, 37, 48] val
43.64 54.55 67.27 87.27000000000001 


,NN-hAIkal
1,43.64
10,54.55
100,67.27
1000,87.27


In [ ]:
get_success_rate(df_snorkel_perceptron_test,"test","NN-hAIkal")

NN-hAIkal [5, 7, 8, 13] test
33.33 46.67 53.33 86.67 


,NN-hAIkal
1,33.33
10,46.67
100,53.33
1000,86.67


In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
RFC_model_core = RandomForestClassifier(n_jobs=-1,n_estimators=100,random_state=32,min_samples_split= 10 , class_weight={0:1.2,1:0.1}) 
RFC_model_haikal = RandomForestClassifier(n_jobs=-1,n_estimators=100,random_state=32,min_samples_split= 10 , class_weight={0:1.2,1:0.1}) 

In [29]:
x_train_gold, y_train_gold = read_data_gold_data_RF()

In [30]:
RFC_model_core.fit(x_train,y_train)

RandomForestClassifier(class_weight={0: 1.2, 1: 0.1}, min_samples_split=10,
                       n_jobs=-1, random_state=32)

In [31]:
RFC_model_haikal.fit(x_train_gold,y_train_gold)

RandomForestClassifier(class_weight={0: 1.2, 1: 0.1}, min_samples_split=10,
                       n_jobs=-1, random_state=32)

In [32]:
x_val_gold = pd.concat( [x_val[scoring_functions],x_ml_classifer[["NNC","TF2"]]],axis=1)
x_test_gold = pd.concat( [x_test[scoring_functions],x_ml_classifer_test[["NNC","TF2"]]],axis=1)

In [33]:
df_core_RFC_val = convert_pred_a(model=RFC_model_core,my_x=x_val[[x_train.columns.to_list()]],my_y=y_val,tag="val")
df_core_RFC_test = convert_pred_a(model=RFC_model_core,my_x=x_test[[x_train.columns.to_list()]],my_y=y_test,tag="test")
df_snorkel_RFC_val = convert_pred_a(model=RFC_model_haikal,my_x=x_val_gold,my_y=y_val,tag="val")
df_snorkel_RFC_test = convert_pred_a(model=RFC_model_haikal,my_x=x_test_gold,my_y=y_test,tag="test")

/home/barradd/miniconda3/envs/ml-module/lib/python3.9/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/barradd/miniconda3/envs/ml-module/lib/python3.9/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/barradd/miniconda3/envs/ml-module/lib/python3.9/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/barradd/miniconda3/envs/ml-module/lib/python3.9/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [34]:
get_success_rate(df_core_RFC_val,"val","RFC")

RFC [23, 30, 33, 45] val
41.82 54.55 60.0 81.82000000000001 


,RFC
1,41.82
10,54.55
100,60.00
1000,81.82


In [35]:
get_success_rate(df_core_RFC_test,"test","RFC")

RFC [2, 7, 9, 13] test
13.33 46.67 60.0 86.67 


,RFC
1,13.33
10,46.67
100,60.00
1000,86.67


In [36]:
get_success_rate(df_snorkel_RFC_val,"val","RFC-hAIkal")

RFC-hAIkal [26, 28, 35, 46] val
47.27 50.91 63.63999999999999 83.64 


,RFC-hAIkal
1,47.27
10,50.91
100,63.64
1000,83.64


In [37]:
get_success_rate(df_snorkel_RFC_test,"test","RFC-hAIkal")

RFC-hAIkal [3, 8, 8, 13] test
20.0 53.33 53.33 86.67 


,RFC-hAIkal
1,20.00
10,53.33
100,53.33
1000,86.67


In [40]:
# get_success_rate(df_core_perceptron_val,"val","NN")
get_success_rate(df_core_RFC_val,"val","RFC")

RFC [23, 30, 33, 45] val
41.82 54.55 60.0 81.82000000000001 


,RFC
1,41.82
10,54.55
100,60.00
1000,81.82


In [41]:
# get_success_rate(df_snorkel_perceptron_val,"val","NN-hAIkal")
get_success_rate(df_snorkel_RFC_val,"val","RFC-hAIkal")

RFC-hAIkal [26, 28, 35, 46] val
47.27 50.91 63.63999999999999 83.64 


,RFC-hAIkal
1,47.27
10,50.91
100,63.64
1000,83.64


In [ ]:
get_success_rate(df_core_perceptron_test,"test","NN")
get_success_rate(df_core_RFC_test,"test","RFC")

NN [6, 7, 8, 13] test
40.0 46.67 53.33 86.67 
RFC [2, 7, 9, 13] test
13.33 46.67 60.0 86.67 


,RFC
1,13.33
10,46.67
100,60.00
1000,86.67


In [42]:
# get_success_rate(df_snorkel_perceptron_test,"test","NN-hAIkal")
get_success_rate(df_snorkel_RFC_test,"test","RFC-hAIkal")


RFC-hAIkal [3, 8, 8, 13] test
20.0 53.33 53.33 86.67 


,RFC-hAIkal
1,20.00
10,53.33
100,53.33
1000,86.67


In [43]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [44]:
### Deep Neural network 
def build_model():
    model = Sequential()
    model.add(Dense(units=40,activation='relu',input_dim=8))
    model.add(Dense(units=16,activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(units=16,activation='relu'))
    model.add(Dense(units=16,activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(units=16,activation='relu'))
    model.add(Dense(units=16,activation='relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(units=16,activation='relu'))
    model.add(Dense(units=16,activation='relu'))
    
    model.add(Dense(units=1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

In [45]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

In [46]:
DNN_model = build_model()

In [ ]:
def read_data_gold_data_TF2() :
    """This funtion takes the data that is already standarized """
    scoring_functions = ['CONSRANK_val', 'CP_HLPL', 'CP_MJ3h', 'DDG_V', 'CP_RMFCA', 'AP_GOAP_DF','TF2','NNC']

    path1 = "../data/snorkel_train_gold_set.csv" 

    df_set_balanced = pd.read_csv(path1)
 
    df_set_balanced.set_index("Conf",inplace=True)
    
     ## Split into Training , validation and Testing Sets 
    x_train = df_set_balanced[~df_set_balanced["idx"].isin(PDB_BM5)]
    y_train = df_set_balanced[~df_set_balanced["idx"].isin(PDB_BM5)]["label_binary"].astype("bool")
    # x_val_bal = df_set_balanced[df_set_balanced["idx"].isin(PDB_BM5)]
    # y_val_bal = df_set_balanced[df_set_balanced["idx"].isin(PDB_BM5)]["label_binary"].astype("bool")
    
    ## Select the features for the sets 
    x_train= x_train[scoring_functions] 

    ## Make a copy for handling better 
    
    x_train = x_train.copy()
    y_train = y_train.copy()
 
    return x_train, y_train


In [47]:
x_val_gold = pd.concat( [x_val[scoring_functions],x_ml_classifer[["NNC","TF2"]]],axis=1)
x_test_gold = pd.concat( [x_test[scoring_functions],x_ml_classifer_test[["NNC","TF2"]]],axis=1)

In [48]:
x_train_gold, y_train_gold = read_data_gold_data_RF()

In [49]:
DNN_model.fit(x=x_train_gold.to_numpy(),
          y=y_train_gold.to_numpy(), 
          epochs=100,
          validation_data=(x_val_gold.to_numpy(), y_val.to_numpy()), 
          verbose=1,
          callbacks=[early_stop]
          )

Epoch 1/100
132022/132022 [==============================] - 456s 3ms/step - loss: 36.2012 - val_loss: 0.6455
Epoch 2/100
132022/132022 [==============================] - 458s 3ms/step - loss: 7.2536 - val_loss: 0.5409
Epoch 3/100
132022/132022 [==============================] - 459s 3ms/step - loss: 0.4190 - val_loss: 0.5922
Epoch 4/100
132022/132022 [==============================] - 459s 3ms/step - loss: 3.8597 - val_loss: 0.5020
Epoch 5/100
132022/132022 [==============================] - 461s 3ms/step - loss: 0.0722 - val_loss: 0.6292
Epoch 6/100
132022/132022 [==============================] - 456s 3ms/step - loss: 281.1795 - val_loss: 0.6344
Epoch 7/100
132022/132022 [==============================] - 462s 3ms/step - loss: 11.1287 - val_loss: 0.5530
Epoch 8/100
132022/132022 [==============================] - 463s 4ms/step - loss: 195.6101 - val_loss: 0.6222
Epoch 9/100
132022/132022 [==============================] - 468s 4ms/step - loss: 0.4888 - val_loss: 0.5259
Epoch 10/100


In [ ]:
for m in x_val_gold.columns.to_list():
    my_na = x_train[x_train[m].isna() == True ]
    print (m,my_na.shape[0])

CONSRANK_val 0
CP_HLPL 0
CP_MJ3h 0
DDG_V 0
CP_RMFCA 0
AP_GOAP_DF 0
RFC 0
NNC 0


In [ ]:
x_val_gold.shape

(157767, 8)

In [ ]:
x_train[x_train.NNC.isna() == True ]

,CONSRANK_val,CP_HLPL,CP_MJ3h,DDG_V,CP_RMFCA,AP_GOAP_DF,RFC,NNC
Conf,,,,,,,,


In [50]:
DNN_model.save(f'../models/TF2_model_for_Ws.h5')

In [59]:
import pickle

In [60]:
def store(b, file_name):
    pickle.dump(b, open(file_name, "wb"))


In [61]:
store(PCR_gold , '../models/PCR_haikal_for_Ws.pickle')
store(RFC_model_haikal , '../models/RFC_haikal_for_Ws.pickle')

In [ ]:
RFC_model_haikal.feature_importances_

array([0.14005727, 0.17810914, 0.0734944 , 0.16473502, 0.19207643,
       0.17126157, 0.0366392 , 0.04362697])

In [ ]:
PCR_gold.coefs_[0].shape

(8, 24)

In [ ]:
PCR_gold.coefs_[0].mean(axis=1)

array([-24.46021681,   1.17478964,   0.08034624,   0.47741979,
        -1.70503827,   0.93513041, -10.86875436,   2.23682232])

In [ ]:
 DNN_model.weights

[<tf.Variable 'dense/kernel:0' shape=(8, 30) dtype=float32, numpy=
 array([[-7.6104843e+01, -1.0926995e+00,  6.8563437e-01, -2.3213940e+00,
         -1.0307978e+00, -9.7981036e-01, -8.3011574e+01,  1.3398010e-01,
         -1.1219685e+00,  1.2900512e-01,  2.5956282e-01,  1.7038462e-01,
          4.9186018e-01, -2.8577831e-01,  1.2503041e+00, -4.0837023e-01,
         -1.0822339e+00,  3.0879599e-01,  5.8575943e-02,  1.6348727e-01,
          2.3884250e-01,  1.8233667e-01, -1.7178452e+00, -4.0716705e-01,
          2.4719672e-01, -8.2018913e+01, -7.4510860e-01, -1.3050075e+00,
          1.6965991e-01, -4.6697807e-01],
        [-1.7634428e-01,  3.2978758e-01, -1.0909324e+00, -1.7471002e+00,
          6.2476987e-01,  1.8678041e-01,  2.5369599e-01,  1.1823199e+00,
          9.9854529e-02, -3.9984566e-01, -7.7376097e-01, -7.2462785e-01,
         -1.2170029e+00,  7.6740915e-01, -2.6981339e+00, -6.1524528e-01,
         -1.5394731e-01,  1.0278294e+00,  1.4286319e-02, -6.6061312e-01,
         -6.326

In [ ]:
DNN_model.input_shape

(None, 8)

In [ ]:
DNN_model.weights[0].get_shape()

TensorShape([8, 30])

In [62]:
DNN_model.weights[0].numpy().mean(axis=1)

array([-1.3212645 , -0.62730306,  0.3123238 , -0.24291953, -0.44588393,
       -0.09557076, -2.0085049 , -1.2761909 ], dtype=float32)

In [63]:
scoring_functions_rfc = ['CONSRANK_val', 'CP_HLPL', 'CP_MJ3h', 'DDG_V', 'CP_RMFCA', 'AP_GOAP_DF','PRC','TF2']


In [65]:
feat_rfc = pd.DataFrame([RFC_model_haikal.feature_importances_] , columns= scoring_functions_rfc ) 
feat_nnc = pd.DataFrame([PCR_gold.coefs_[0].mean(axis=1)] , columns= scoring_functions_rfc )
feat_dnn = pd.DataFrame([DNN_model.weights[0].numpy().mean(axis=1)] , columns= scoring_functions_rfc )  

In [66]:
print ( pd.concat( [ feat_rfc, feat_nnc,feat_dnn], keys=['RFC','PRC','TF2']  ).to_csv() ) 

,,CONSRANK_val,CP_HLPL,CP_MJ3h,DDG_V,CP_RMFCA,AP_GOAP_DF,PRC,TF2
RFC,0,0.14005727079365152,0.17810913555857397,0.0734943950476807,0.16473502371583007,0.19207642835705954,0.17126157363234124,0.03663920296381242,0.04362696993105059
PRC,0,-29.451714106646293,-3.209337420624555,0.12833598891395404,0.6884689638496502,-0.532188995810691,1.7510614579536277,-2.326788126457157,-2.7415447901516505
TF2,0,-1.3212645053863525,-0.6273030638694763,0.31232380867004395,-0.24291953444480896,-0.44588392972946167,-0.09557075798511505,-2.008504867553711,-1.2761908769607544



In [69]:
pd.concat( [ feat_rfc, feat_nnc,feat_dnn], keys=['RFC','PRC','TF2']  ).T.to_csv()

',RFC,PRC,TF2\n,0,0,0\nCONSRANK_val,0.14005727079365152,-29.451714106646293,-1.3212645053863525\nCP_HLPL,0.17810913555857397,-3.209337420624555,-0.6273030638694763\nCP_MJ3h,0.0734943950476807,0.12833598891395404,0.31232380867004395\nDDG_V,0.16473502371583007,0.6884689638496502,-0.24291953444480896\nCP_RMFCA,0.19207642835705954,-0.532188995810691,-0.44588392972946167\nAP_GOAP_DF,0.17126157363234124,1.7510614579536277,-0.09557075798511505\nPRC,0.03663920296381242,-2.326788126457157,-2.008504867553711\nTF2,0.04362696993105059,-2.7415447901516505,-1.2761908769607544\n'